## Homework 5 Questions

Same drill as Homework 3, the following questions are intended to get you familiarized with the [provided dataset](https://docs.google.com/spreadsheets/d/1tcqGAlaBOd9L9Lhb9u9_nUg9dSRY1usF80l2RNKiQN0/edit?usp=sharing). By answering these questions, it will hopefully give insights on how to complete the actual task (slide deck generation for the CEO).

Please answer each of the questions below in the space provided. Copy the noteook, and when you're done, submit as an ipynb file. This will be graded on correctness. Please comment your code to help the grader figure out what you're doing. If you do a hypothesis test, please report all relevant p-values in the text field.

Imports and data loading:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import pprint
import scipy as scipy

In [3]:
path = "./drive/MyDrive/cmsc320/HW5/GPA_Dataset.csv"
df = pd.read_csv(path)

Data cleaning:

1. Which was the most predictive factor for GPA? i.e., which factor had the most "power" over predicting the GPA?

Your answer:

In [ ]:
# numericals:
# ["At what age did you start programming?", "How many credits did you take last semester?", "What is your average screentime?", "What is your average daily time on Tik Tok or Tik Tok equivalents?", "How many hours of sleep do you get a night?"]
target = "What was your GPA LAST SEMESTER?"

numericals = ["At what age did you start programming?", "How many credits did you take last semester?", "What is your average screentime?", "What is your average daily time on Tik Tok or Tik Tok equivalents?", "How many hours of sleep do you get a night?", "What was your GPA THE SEMESTER BEFORE THAT?" ]
categoricals = list(set(list(df.columns[:-2]) + list([df.columns[-1]])) - set(numericals))

print("R^2 values after running Random Forest Regression trying to predict GPA last semester using some other feature\n-------------------------------------------------------------------------------------------------------")
print("%-80s:\tR^2"%"column")
print()
for i, lab in enumerate(numericals):
  null_filter1 = df[lab].isnull()
  filter = np.logical_not(np.logical_or(null_filter, null_filter1))
  gpa_target = df[target][filter].to_list()
  X = np.array(df[lab][filter].to_list()).reshape(-1,1)
  reg = RandomForestRegressor(max_depth=2, random_state=0).fit(X, gpa_target)
  print("%-80s:\t %f"%(lab, reg.score(X, gpa_target)))

for i, lab in enumerate(categoricals):
  null_filter1 = df[lab].isnull()
  filter = np.logical_not(np.logical_or(null_filter, null_filter1))
  gpa_target = df[target][filter].to_list()
  X = np.array(pd.get_dummies(df[lab][filter]))
  reg = RandomForestRegressor(max_depth=2, random_state=0).fit(X, gpa_target)
  print("%-80s:\t %f"%(lab, reg.score(X, gpa_target)))

print("\n\nAs can be seen, the strongest predictor of the GPA last semester was the GPA 2 semesters prior.\nWith an R^2 value of 0.643, it had the highest R^2 score out of any of the other features")

R^2 values after running Random Forest Regression trying to predict GPA last semester using some other feature
-------------------------------------------------------------------------------------------------------
column                                                                          :	R^2

At what age did you start programming?                                          :	 0.103036
How many credits did you take last semester?                                    :	 0.076171
What is your average screentime?                                                :	 0.172657
What is your average daily time on Tik Tok or Tik Tok equivalents?              :	 0.039582
How many hours of sleep do you get a night?                                     :	 0.088473
What was your GPA THE SEMESTER BEFORE THAT?                                     :	 0.642563
Do you take notes in classes?                                                   :	 0.079039
Timestamp                                             

2. Which factor(s) didn't matter in predicting the GPA?

Your answer:

In [ ]:
print("Using the same table above, the feature that had the lowest R^2 score when predicting GPA last semester was whether the participant had attended private school.\nThat is, the \"Have you at any point attended private schoo?\" column had the lowest R^2 when the Random Forest regression was ran on it\ncompared to the other factors")

Using the same table above, the feature that had the lowest R^2 score when predicting GPA last semester was whether the participant had attended private school.
That is, the "Have you at any point attended private schoo?" column had the lowest R^2 when the Random Forest regression was ran on it
compared to the other factors


3. Besides the GPA information, which two questions had the highest correlation?

Your answer:

In [ ]:
numericals = ["At what age did you start programming?", "How many credits did you take last semester?", "What is your average screentime?", "What is your average daily time on Tik Tok or Tik Tok equivalents?", "How many hours of sleep do you get a night?", "What was your GPA THE SEMESTER BEFORE THAT?"]
categoricals = list(set(list(df.columns[:-2]) + list([df.columns[-1]])) - set(numericals))

nice_display = lambda y: lambda x: x[:x.find(".")] + x[x.find("."):x.find(".")+y]
nice_display = nice_display(4)
corrs = []

print("Correlations of each pair of columns. Numerical to Numerical uses pearson correlation coefficient and Categorical to Categorical uses Cramer.\nCategorical to Numerical doesn't make sense so it's not considered\n\n")

print("Correlation of Numericals:\n\n")
for i, lab1 in enumerate(numericals[:-1]):
  print("correlation of %-60s AND:\n------------------------"%("\""+lab1+"\""))
  for j, lab2 in enumerate(numericals):
    if j <= i:
      continue
    null_filter1 = df[lab1].isnull()
    null_filter2 = df[lab2].isnull()
    filter = np.logical_not(np.logical_or(null_filter1, null_filter2))
    corr = np.corrcoef(np.array(df[lab1][filter]), np.array(df[lab2][filter]))[0,1]
    corrs.append(corr)
    print("%-70s\t:%13s"%("\""+lab2+"\"", nice_display(str(corr))))
  print()

def make_confusion_matrix(df, col1, col2, filter):
  c1 = df[col1][filter].astype(str)
  c2 = df[col2][filter].astype(str)
  vals1 = np.unique(c1)
  vals2 = np.unique(c2)
  conf = np.zeros((len(vals1), len(vals2)))
  for i, v1 in enumerate(vals1):
    for j, v2 in enumerate(vals2):
      conf[i,j] = np.sum(np.logical_and(np.array(c1 == v1), np.array(c2 == v2)))

  return conf+1


print("\nCorrelation of Categoricals:\n\n")
for i, lab1 in enumerate(categoricals[:-1]):
  print("correlation of %-60s AND:\n------------------------"%("\"" + lab1 + "\""))
  for j, lab2 in enumerate(categoricals):
    if j <= i:
      continue
    null_filter1 = df[lab1].isnull()
    null_filter2 = df[lab2].isnull()
    filter = np.logical_not(np.logical_or(null_filter1, null_filter2))
    conf_matrix = make_confusion_matrix(df, lab1, lab2, filter)
    if 0 in conf_matrix:
      continue
    r,c = conf_matrix.shape
    sum1 = scipy.stats.chi2_contingency(conf_matrix).statistic
    corr = np.sqrt(((sum1)/(r*c))/min(r-1,c-1))
    corrs.append(corr)
    print("%-70s\t:%13s"%("\""+lab2+"\"", nice_display(str(corr))))
  print()

print("\n*Note that it doesn't make sense to compute the correlation between categorical data and numerical data.\nFrom the table, we can see that the most correlated questions were studying with friends and attending private school\nThat is, the correlation of \"Have you at any point attended private school?\" and \"Do you study with friends?\" had the highest correlation at %f"%(max(corrs)))

    # corr = np.corrcoef(np.array(df[lab1][filter]), np.array(df[lab2][filter]))[0,1]
    # print("\"%-70s\"\t:%13s"%(lab2, nice_display(str(corr))))

# chi-squared - categorical to categorical
# - Cramer V
# correlation - numerical to numrerical
# ANOVA - categorical to numerical

Correlations of each pair of columns. Numerical to Numerical uses pearson correlation coefficient and Categorical to Categorical uses Cramer.
Categorical to Numerical doesn't make sense so it's not considered


Correlation of Numericals:


correlation of "At what age did you start programming?"                     AND:
------------------------
"How many credits did you take last semester?"                        	:       -0.260
"What is your average screentime?"                                    	:       -0.060
"What is your average daily time on Tik Tok or Tik Tok equivalents?"  	:        0.027
"How many hours of sleep do you get a night?"                         	:       -0.119
"What was your GPA THE SEMESTER BEFORE THAT?"                         	:       -0.218

correlation of "How many credits did you take last semester?"               AND:
------------------------
"What is your average screentime?"                                    	:        0.016
"What is your average daily tim

4. Which one had more impact on students' quality of sleep, the amount of screen time or course load?

Your answer:

In [ ]:
# numericals:
# ["At what age did you start programming?", "How many credits did you take last semester?", "What is your average screentime?", "What is your average daily time on Tik Tok or Tik Tok equivalents?", "How many hours of sleep do you get a night?"]
target = "How many hours of sleep do you get a night?"

cols = ["What is your average screentime?", "How many credits did you take last semester?"]

print("R^2 values after running Random Forest Regression trying to predict sleep using screen time vs using credits\n-------------------------------------------------------------------------------------------------------")
print("%-80s:\tR^2"%"column")
print()
for i, lab in enumerate(cols):
  null_filter1 = df[lab].isnull()
  filter = np.logical_not(np.logical_or(null_filter, null_filter1))
  gpa_target = df[target][filter].to_list()
  X = np.array(df[lab][filter].to_list()).reshape(-1,1)
  reg = RandomForestRegressor(max_depth=2, random_state=0).fit(X, gpa_target)
  print("%-80s:\t %f"%(lab, reg.score(X, gpa_target)))

# for i, lab in enumerate(categoricals):
#   null_filter1 = df[lab].isnull()
#   filter = np.logical_not(np.logical_or(null_filter, null_filter1))
#   gpa_target = df[target][filter].to_list()
#   X = np.array(pd.get_dummies(df[lab][filter]))
#   reg = RandomForestRegressor(max_depth=2, random_state=0).fit(X, gpa_target)
#   print("%-80s:\t %f"%(lab, reg.score(X, gpa_target)))

print("\n\nAs can be seen, screentime is a stronger predictor of sleep than course load because its R^2 value after running a \nRandom Forest Regression is higher. Predicting sleep with screentime has an R^2 value is 0.21 as opposed to 0.12 for course load")

R^2 values after running Random Forest Regression trying to predict sleep using screen time vs using credits
-------------------------------------------------------------------------------------------------------
column                                                                          :	R^2

What is your average screentime?                                                :	 0.210990
How many credits did you take last semester?                                    :	 0.120267


As can be seen, screentime is a stronger predictor of sleep than course load because its R^2 value after running a 
Random Forest Regression is higher. Predicting sleep with screentime has an R^2 value is 0.21 as opposed to 0.12 for course load


5. Does having a parent (or more) in tech have impact on the starting age for programming?

Your answer:

In [32]:
def make_confusion_matrix(df, col1, col2, filter):
  c1 = df[col1][filter].astype(str)
  c2 = df[col2][filter].astype(str)
  vals1 = np.unique(c1)
  vals2 = np.unique(c2)
  conf = np.zeros((len(vals1), len(vals2)))
  for i, v1 in enumerate(vals1):
    for j, v2 in enumerate(vals2):
      conf[i,j] = np.sum(np.logical_and(np.array(c1 == v1), np.array(c2 == v2)))

  return conf+1

lab1 = "Do you have a parent in tech?"
lab2 = "At what age did you start programming?"
df.loc[:, lab1] = df[lab1].apply(lambda x: 0 if x == "No parents" else 1)

null_filter = df[lab1].isnull()
null_filter1 = df[lab2].isnull()
filter = np.logical_not(np.logical_or(null_filter, null_filter1))

conf_matrix = make_confusion_matrix(df, lab1, lab2, filter)
test_stat, p_val, dof, expected_freq = scipy.stats.chi2_contingency(conf_matrix)

print("Test:\nNull:\t\tparent in tech and programming start are independent\nAlternate:\tparent in tech and programming start are NOT independent, reject Null if p-val < 0.06")
print("------------------------------------------------------\n")
print("p-value: %f"%(p_val))
print("Since the p-value is not <0.06, the null is NOT rejected, and so the age of programming and whether a parent is in tech are indpendent factors")
df = pd.read_csv(path)

Test:
Null:		parent in tech and programming start are independent
Alternate:	parent in tech and programming start are NOT independent, reject Null if p-val < 0.06
------------------------------------------------------

p-value: 0.764372
Since the p-value is not <0.06, the null is NOT rejected, and so the age of programming and whether a parent is in tech are indpendent factors


6. Assume that the students who gave response to this survey have a consistent studying habit. Based on that assumption, does studying with friends help with grades? What about note taking?



Your answer:

In [33]:
def make_confusion_matrix(df, col1, col2, filter):
  c1 = df[col1][filter].astype(str)
  c2 = df[col2][filter].astype(str)
  vals1 = np.unique(c1)
  vals2 = np.unique(c2)
  conf = np.zeros((len(vals1), len(vals2)))
  for i, v1 in enumerate(vals1):
    for j, v2 in enumerate(vals2):
      conf[i,j] = np.sum(np.logical_and(np.array(c1 == v1), np.array(c2 == v2)))

  return conf+1

target_lab = "What was your GPA LAST SEMESTER?"
labs = ["Do you study with friends?", "Do you study with friends?"]

null_filter = df[target_lab].isnull()
null_filter1 = df[labs[0]].isnull()
filter = np.logical_not(np.logical_or(null_filter, null_filter1))
conf_matrix = make_confusion_matrix(df, labs[0], target_lab, filter)
test_stat, p_val0, dof, expected_freq = scipy.stats.chi2_contingency(conf_matrix)

null_filter = df[target_lab].isnull()
null_filter1 = df[labs[1]].isnull()
filter = np.logical_not(np.logical_or(null_filter, null_filter1))
conf_matrix = make_confusion_matrix(df, labs[1], target_lab, filter)
test_stat, p_val1, dof, expected_freq = scipy.stats.chi2_contingency(conf_matrix)

print("Test:\nNull:\t\tstudying with friends/note-taking and grades are independent\nAlternate:\tstudying with friends/note-taking and grades are NOT independent, reject Null if p-val < 0.06")
print("------------------------------------------------------\n")
print("p-value for studying with friends: %f"%(p_val0))
print("p-value for taking notes in class: %f"%(p_val1))
print("from the p-values, we have that studying with friends is independent of grades, and note-taking and grades ARE dependent")
df = pd.read_csv(path)

Test:
Null:		studying with friends/note-taking and grades are independent
Alternate:	studying with friends/note-taking and grades are NOT independent, reject Null if p-val < 0.06
------------------------------------------------------

p-value for studying with friends: 0.548900
p-value for taking notes in class: 0.049890
from the p-values, we have that studying with friends is independent of grades, and note-taking and grades ARE dependent
